In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow as tf
from tensorflow.keras import layers

# Define the V-Net model
def VNet(input_shape):
    # Encoder
    inputs = tf.keras.Input(shape=input_shape)
    conv1 = layers.Conv3D(16, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv3D(16, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling3D(pool_size=(2, 2, 2))(conv1)

    conv2 = layers.Conv3D(32, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv3D(32, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling3D(pool_size=(2, 2, 2))(conv2)

    conv3 = layers.Conv3D(64, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv3D(64, 3, activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling3D(pool_size=(2, 2, 2))(conv3)

    # Decoder
    conv4 = layers.Conv3D(64, 3, activation='relu', padding='same')(pool3)
    conv4 = layers.Conv3D(64, 3, activation='relu', padding='same')(conv4)
    up1 = layers.UpSampling3D(size=(2, 2, 2))(conv4)
    up1 = layers.concatenate([conv3, pad(up1, conv3)], axis=-1)

    conv5 = layers.Conv3D(32, 3, activation='relu', padding='same')(up1)
    conv5 = layers.Conv3D(32, 3, activation='relu', padding='same')(conv5)
    up2 = layers.UpSampling3D(size=(2, 2, 2))(conv5)
    up2 = layers.concatenate([conv2, pad(up2, conv2)], axis=-1)

    conv6 = layers.Conv3D(16, 3, activation='relu', padding='same')(up2)
    conv6 = layers.Conv3D(16, 3, activation='relu', padding='same')(conv6)
    up3 = layers.UpSampling3D(size=(2, 2, 2))(conv6)
    up3 = layers.concatenate([conv1, pad(up3, conv1)], axis=-1)

    outputs = layers.Conv3D(1, 1, activation='sigmoid')(up3)

    # Create the model
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# Define the padding function
def pad(inputs, ref):
    target_shape = ref.shape[1:-1]
    inputs_shape = inputs.shape[1:-1]

    pad_start = [(t - i) // 2 for t, i in zip(target_shape, inputs_shape)]
    pad_end = [t - i - s for t, i, s in zip(target_shape, inputs_shape, pad_start)]
    paddings = [(0, 0)] + [(s, e) for s, e in zip(pad_start, pad_end)] + [(0, 0)]

    return tf.pad(inputs, paddings)